## PEGASUS Local

In [1]:
import sys
sys.path.append('..')

import torch
from parser.parser import Parser

from scorer import Score

from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_summarizer'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Getting the token and downloading the model

In [2]:
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

### Getting response

In [3]:
def get_response(input_text):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=1024, return_tensors="pt").to(torch_device)
  gen_out = model.generate(**batch,max_length=128,num_beams=5, num_return_sequences=1, do_sample = True, temperature=1.5)
  output_text = tokenizer.batch_decode(gen_out, skip_special_tokens=True)
  return output_text


### Getting the Summary

In [10]:
paperName = "2310.10737"

content = Parser(pdfFile = "../papers/" + paperName +".pdf")

pegasusSummary = get_response(content)[0]

print(pegasusSummary)

A large, flexible class of long, high-redundancy error correcting codes, GRAND, can be efficiently decoded with guessing random additive noise dec oding (GRAND). Grand is a recently developed family of code-agnostic decoding algorithms that can accurately decode long, high- redundancy codes. It can be used to decode long, high-redundancy codes while LDPC codes are used to decode long, high-redundancy codes.


In [11]:
goldenSummary = ""

with open ( "../papers/" + paperName + "_ABSTRACT.txt") as f :
    goldenSummary = f.read()
    
score = Score(trueSummary = goldenSummary, predSummary = pegasusSummary)

In [12]:
for key, value in score.rougeScore().items():
    print(f"Criteria:{key}, Score:{value}" )

Criteria:rouge1, Score:Score(precision=0.703125, recall=0.17928286852589642, fmeasure=0.2857142857142857)
Criteria:rouge2, Score:Score(precision=0.2857142857142857, recall=0.072, fmeasure=0.11501597444089456)
Criteria:rougeL, Score:Score(precision=0.515625, recall=0.13147410358565736, fmeasure=0.2095238095238095)
